In [14]:
def open_left(x, alpha, beta):
    if x <= alpha:
        return 1 

    elif alpha < x < beta:
        return (beta - x)/ (beta - alpha)

    else:
        return 0

In [15]:
def open_right(x, a, b):
    if x>= b:
        return 1
    elif x <= a:
        return 0
    else:
        return (x-a)/(b-a)

In [16]:
def triangular(x, a , b, c):
    return max(min((x-a)/(b-a), (c-x)/(c-b)),0)

In [17]:
def fuzzifying(x):
    '''
    input x -> need to find the membership value for x
    return -> which function it touches and what's the membership value
    '''
    NL, NM, NS, ZE, PS, PM, PL = 0, 0, 0, 0, 0, 0, 0
    
    if x >= 0 and x <= 60:
        NL = open_left(x, 30, 60)
        
    if x >= 30 and x <= 90:
        NM = triangular(x, 30, 60, 90)
        
    if x>= 60 and x <= 120:
        NS = triangular(x, 60, 90, 120)
        
    if x >= 90 and x <= 150:
        ZE = triangular(x, 90, 120, 150)
        
    if x>= 120 and x <= 180:
        PS = triangular(x, 120, 150, 180)
        
    if x >= 150 and x <= 210:
        PM = triangular(x, 150, 180, 210)
        
    if x>= 180 and x <= 210:
        PL = open_right(x, 180, 210)
        
    return  NL, NM, NS, ZE, PS, PM, PL
        

In [31]:
speed_difference = 110
NLSD, NMSD, NSSD, ZESD, PSSD, PMSD, PLSD = fuzzifying(speed_difference)
print(NLSD, NMSD, NSSD, ZESD, PSSD, PMSD, PLSD)

0 0 0.3333333333333333 0.6666666666666666 0 0 0


In [32]:
accelaration = 115
NLAC, NMAC, NSAC, ZEAC, PSAC, PMAC, PLAC = fuzzifying(accelaration)
print(NLAC, NMAC, NSAC, ZEAC, PSAC, PMAC, PLAC)

0 0 0.16666666666666666 0.8333333333333334 0 0 0


## Rules
- R1: if SD is NL and A is ZE then TC is PL
- R2: if SD is ZE and A is NL then TC is PL
- R3: if SD is NM and A is ZE then TC is PM
- R4: if SD is NS and A is PS then TC is PS
- R5: if SD is PS and A is NS then TC is NS
- R6: if SD is PL and A is ZE then TC is NL
- R7: if SD is ZE and A is NS then TC is PS
- R8: if SD is ZE and A is NM then TC is PM


In [20]:
def rule_comparison(TC1, TC2):
    TC = 0
    if TC1>TC2 and TC1 !=0 and TC2 !=0:
        TC = TC2
    else:
        TC = TC1
    
    if TC1 == 0 and TC2 !=0:
        TC = TC2
        
    if TC2 == 0 and TC1 !=0:
        TC = TC1
        
    return TC

In [21]:
def rule(NLSD, NMSD, NSSD, ZESD, PSSD, PMSD, PLSD, NLAC, NMAC, NSAC, ZEAC, PSAC, PMAC, PLAC):
    # Rule 1
    PLTC1 = min(NLSD, ZEAC)
    # Rule 2
    PLTC2 = min(ZESD, NLAC)
    PLTC = rule_comparison(PLTC1, PLTC2)
    
    
    PMTC1 = min(NMSD,ZEAC)
    PMTC2 = min(ZESD,NMAC)
    PMTC = rule_comparison(PMTC1, PMTC2)
    
    PSTC1 = min(NSSD,PSAC)
    PSTC2 = min(ZESD,NSAC)
    PSTC = rule_comparison(PSTC1, PSTC2)
    
    NSTC = min(PSSD,NSAC)
    NLTC = min(PLSD,ZEAC)
    
    return PLTC, PMTC, PSTC, NSTC, NLTC


In [33]:
PLTC, PMTC, PSTC, NSTC, NLTC = rule(NLSD,NMSD,NSSD,ZESD,PSSD,PMSD,PLSD,NLAC,NMAC,NSAC,ZEAC,PSAC,PMAC,PLAC)
print(PLTC, PMTC, PSTC, NSTC, NLTC)

0 0 0.16666666666666666 0 0


In [23]:
def areaTR(mu, a,b,c):
    x1 = mu*(b-a) + a
    x2 = c - mu*(c-b)
    d1 = (c-a); d2 = x2-x1
    a = (1/2)*mu*(d1 + d2)
    return a # Returning area

In [24]:
def areaOL(mu, alpha, beta):
    xOL = beta -mu*(beta - alpha)
    return 1/2*mu*(beta+ xOL), beta/2

In [25]:
def areaOR(mu, alpha, beta):
    xOR = (beta - alpha)*mu + alpha
    aOR = (1/2)*mu*((240 - alpha) + (240 -xOR))
    return aOR, (240 - alpha)/2 + alpha

In [26]:
def defuzzyfication(PLTC, PMTC, PSTC, NSTC, NLTC):
    areaPL = 0; areaPM = 0; areaPS = 0; areaNS = 0; areaNL = 0;
    cPL = 0; cPM = 0; cPS = 0; cNS = 0; cNL = 0;

    if PLTC != 0:
        #areaPL, cPL = areaOR(PLTC, 180, 210)
        areaPL, cPL = areaOR(PLTC, 180, 210)
                
    if PMTC != 0:
        areaPM = areaTR(PMTC, 150, 180, 210)
        cPM = 180
    
    if PSTC != 0:
        areaPS = areaTR(PSTC, 120, 150, 180)
        cPS = 150
          
    if NSTC != 0:
        areaNS = areaTR(NSTC, 60, 90, 120)
        cNS = 90
        
    if NLTC !=0:
        areaNL, cNL = areaOL(NLTC, 30, 60)
        
    numerator = areaPL*cPL + areaPM*cPM + areaPS*cPS + areaNS*cNS + areaNL*cNL
    denominator = areaPL + areaPM + areaPS + areaNS + areaNL
    if denominator ==0:
        print("No rules exist to give the result")
        return 0
    else:
        crispOutput = numerator/denominator
        return crispOutput

In [34]:
crispOutputFinal = defuzzyfication(PLTC, PMTC, PSTC, NSTC, NLTC)
crispOutputFinal

150.0